# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 15 2022 3:13PM,239685,10,Enova-10144,Emerginnova,Released
1,Jun 15 2022 3:13PM,239685,10,Enova-10145,Emerginnova,Released
2,Jun 15 2022 3:06PM,239684,10,CTF0011271,"Citieffe, Inc.",Released
3,Jun 15 2022 3:06PM,239684,10,CTF0011272,"Citieffe, Inc.",Released
4,Jun 15 2022 3:06PM,239684,10,CTF0011273,"Citieffe, Inc.",Released
5,Jun 15 2022 3:06PM,239676,10,MSP211293,"Methapharm, Inc.",Released
6,Jun 15 2022 3:06PM,239676,10,MSP211294,"Methapharm, Inc.",Released
7,Jun 15 2022 3:06PM,239676,10,MSP211295,"Methapharm, Inc.",Released
8,Jun 15 2022 3:06PM,239676,10,MSP211296,"Methapharm, Inc.",Released
9,Jun 15 2022 3:06PM,239676,10,MSP211297,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,239680,Released,1
46,239682,Released,9
47,239683,Released,1
48,239684,Released,3
49,239685,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239680,NaN,NaN,1.0
239682,NaN,NaN,9.0
239683,NaN,NaN,1.0
239684,NaN,NaN,3.0
239685,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239590,0.0,0.0,1.0
239595,0.0,1.0,0.0
239609,3.0,5.0,1.0
239614,0.0,0.0,1.0
239622,25.0,7.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239590,0,0,1
239595,0,1,0
239609,3,5,1
239614,0,0,1
239622,25,7,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239590,0,0,1
1,239595,0,1,0
2,239609,3,5,1
3,239614,0,0,1
4,239622,25,7,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239590,,,1
1,239595,,1,
2,239609,3,5,1
3,239614,,,1
4,239622,25,7,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 15 2022 3:13PM,239685,10,Emerginnova
2,Jun 15 2022 3:06PM,239684,10,"Citieffe, Inc."
5,Jun 15 2022 3:06PM,239676,10,"Methapharm, Inc."
31,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.
32,Jun 15 2022 2:44PM,239682,10,ISDIN Corporation
41,Jun 15 2022 2:42PM,239680,10,Yusen – 3D Matrix
42,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation
79,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd"
93,Jun 15 2022 2:18PM,239677,10,"Methapharm, Inc."
98,Jun 15 2022 2:17PM,239675,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 15 2022 3:13PM,239685,10,Emerginnova,,,2
1,Jun 15 2022 3:06PM,239684,10,"Citieffe, Inc.",,,3
2,Jun 15 2022 3:06PM,239676,10,"Methapharm, Inc.",,,26
3,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.,,,1
4,Jun 15 2022 2:44PM,239682,10,ISDIN Corporation,,,9
5,Jun 15 2022 2:42PM,239680,10,Yusen – 3D Matrix,,,1
6,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation,,,37
7,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd",,,14
8,Jun 15 2022 2:18PM,239677,10,"Methapharm, Inc.",,,5
9,Jun 15 2022 2:17PM,239675,10,ISDIN Corporation,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 3:13PM,239685,10,Emerginnova,2,,
1,Jun 15 2022 3:06PM,239684,10,"Citieffe, Inc.",3,,
2,Jun 15 2022 3:06PM,239676,10,"Methapharm, Inc.",26,,
3,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.,1,,
4,Jun 15 2022 2:44PM,239682,10,ISDIN Corporation,9,,
5,Jun 15 2022 2:42PM,239680,10,Yusen – 3D Matrix,1,,
6,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation,37,,
7,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd",14,,
8,Jun 15 2022 2:18PM,239677,10,"Methapharm, Inc.",5,,
9,Jun 15 2022 2:17PM,239675,10,ISDIN Corporation,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 3:13PM,239685,10,Emerginnova,2,,
1,Jun 15 2022 3:06PM,239684,10,"Citieffe, Inc.",3,,
2,Jun 15 2022 3:06PM,239676,10,"Methapharm, Inc.",26,,
3,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.,1,,
4,Jun 15 2022 2:44PM,239682,10,ISDIN Corporation,9,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 3:13PM,239685,10,Emerginnova,2.0,NaN,NaN
1,Jun 15 2022 3:06PM,239684,10,"Citieffe, Inc.",3.0,NaN,NaN
2,Jun 15 2022 3:06PM,239676,10,"Methapharm, Inc.",26.0,NaN,NaN
3,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.,1.0,NaN,NaN
4,Jun 15 2022 2:44PM,239682,10,ISDIN Corporation,9.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 3:13PM,239685,10,Emerginnova,2.0,0.0,0.0
1,Jun 15 2022 3:06PM,239684,10,"Citieffe, Inc.",3.0,0.0,0.0
2,Jun 15 2022 3:06PM,239676,10,"Methapharm, Inc.",26.0,0.0,0.0
3,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.,1.0,0.0,0.0
4,Jun 15 2022 2:44PM,239682,10,ISDIN Corporation,9.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3834656,191.0,35.0,25.0
15,718938,8.0,0.0,0.0
16,239651,1.0,2.0,0.0
18,479271,2.0,0.0,0.0
19,1677483,22.0,5.0,3.0
20,1677653,43.0,75.0,15.0
21,239595,0.0,1.0,0.0
22,718977,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3834656,191.0,35.0,25.0
1,15,718938,8.0,0.0,0.0
2,16,239651,1.0,2.0,0.0
3,18,479271,2.0,0.0,0.0
4,19,1677483,22.0,5.0,3.0
5,20,1677653,43.0,75.0,15.0
6,21,239595,0.0,1.0,0.0
7,22,718977,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,191.0,35.0,25.0
1,15,8.0,0.0,0.0
2,16,1.0,2.0,0.0
3,18,2.0,0.0,0.0
4,19,22.0,5.0,3.0
5,20,43.0,75.0,15.0
6,21,0.0,1.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,191.0
1,15,Released,8.0
2,16,Released,1.0
3,18,Released,2.0
4,19,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,25.0,0.0,0.0,0.0,3.0,15.0,0.0,0.0
Executing,35.0,0.0,2.0,0.0,5.0,75.0,1.0,0.0
Released,191.0,8.0,1.0,2.0,22.0,43.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,25.0,0.0,0.0,0.0,3.0,15.0,0.0,0.0
1,Executing,35.0,0.0,2.0,0.0,5.0,75.0,1.0,0.0
2,Released,191.0,8.0,1.0,2.0,22.0,43.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,25.0,0.0,0.0,0.0,3.0,15.0,0.0,0.0
1,Executing,35.0,0.0,2.0,0.0,5.0,75.0,1.0,0.0
2,Released,191.0,8.0,1.0,2.0,22.0,43.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()